In [5]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [6]:
%pdb off

**Source: **[In-, out- and net-migration estimates, by geographic regions of origin and destination, *Terminated*](http://www5.statcan.gc.ca/cansim/a26?lang=eng&retrLang=eng&id=1110030&paSer=&pattern=&stByVal=1&p1=1&p2=-1&tabMode=dataTable&csid=)

In [7]:
flowData = pd.read_csv('../TableD_01110030-eng.csv')
flowData.head()

,Ref_Date,GEO,Geographical classification,GEODEST,Geographical classification.1,MIGMOVE,Vector,Coordinate,Value
0,2000,"St. John's, Newfoundland and Labrador",10001,"Non-census metropolitan area, Newfoundland and...",NaN,In-migration,v21575997,1.2.1,3013.000
1,2001,"St. John's, Newfoundland and Labrador",10001,"Non-census metropolitan area, Newfoundland and...",NaN,In-migration,v21575997,1.2.1,3476.000
2,2002,"St. John's, Newfoundland and Labrador",10001,"Non-census metropolitan area, Newfoundland and...",NaN,In-migration,v21575997,1.2.1,3280.000
3,2003,"St. John's, Newfoundland and Labrador",10001,"Non-census metropolitan area, Newfoundland and...",NaN,In-migration,v21575997,1.2.1,3396.000
4,2004,"St. John's, Newfoundland and Labrador",10001,"Non-census metropolitan area, Newfoundland and...",NaN,In-migration,v21575997,1.2.1,3278.000


In [8]:
dropCols = ['Geographical classification',
            'Geographical classification.1',
            'Coordinate']
flowData = flowData.drop(dropCols, axis=1)
flowData.head(10)

,Ref_Date,GEO,GEODEST,MIGMOVE,Vector,Value
0,2000,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",In-migration,v21575997,3013.000
1,2001,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",In-migration,v21575997,3476.000
2,2002,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",In-migration,v21575997,3280.000
3,2003,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",In-migration,v21575997,3396.000
4,2004,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",In-migration,v21575997,3278.000
5,2005,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",In-migration,v21575997,3426.000
6,2006,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",In-migration,v21575997,3229.000
7,2007,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",In-migration,v21575997,3403.000
8,2008,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",In-migration,v21575997,3295.000
9,2009,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",In-migration,v21575997,3272.000


In [9]:
# Filter for only the most recent data
flowData2011 = flowData[flowData['Ref_Date'] == 2011].drop('Ref_Date', axis=1).reset_index(drop=True)
flowData2011.head()

,GEO,GEODEST,MIGMOVE,Vector,Value
0,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",In-migration,v21575997,3402.000
1,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",Out-migration,v21575998,1842.000
2,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Newfoundland and...",Net-migration,v21575999,1560.000
3,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Prince Edward Is...",In-migration,v21576000,28.000
4,"St. John's, Newfoundland and Labrador","Non-census metropolitan area, Prince Edward Is...",Out-migration,v21576001,47.000


In [10]:
flowData2011['Value'] = flowData2011['Value'].convert_objects(convert_numeric=True)

In [11]:
flowData2011_cma = flowData2011[~flowData2011['GEODEST'].str.contains('Non-census')]
flowData2011_cma = flowData2011_cma[~flowData2011_cma['GEO'].str.contains('Non-census')]
flowData2011_cma

,GEO,GEODEST,MIGMOVE,Vector,Value
6,"St. John's, Newfoundland and Labrador","Halifax, Nova Scotia",In-migration,v21576003,392
7,"St. John's, Newfoundland and Labrador","Halifax, Nova Scotia",Out-migration,v21576004,254
8,"St. John's, Newfoundland and Labrador","Halifax, Nova Scotia",Net-migration,v21576005,138
12,"St. John's, Newfoundland and Labrador","Moncton, New Brunswick",In-migration,v42228405,67
13,"St. John's, Newfoundland and Labrador","Moncton, New Brunswick",Out-migration,v42228406,37
14,"St. John's, Newfoundland and Labrador","Moncton, New Brunswick",Net-migration,v42228407,30
15,"St. John's, Newfoundland and Labrador","Saint John, New Brunswick",In-migration,v21576009,41
16,"St. John's, Newfoundland and Labrador","Saint John, New Brunswick",Out-migration,v21576010,25
17,"St. John's, Newfoundland and Labrador","Saint John, New Brunswick",Net-migration,v21576011,16
21,"St. John's, Newfoundland and Labrador","Saguenay, Quebec",In-migration,v21576015,1
